In [3]:
suppressMessages(suppressWarnings(library(tidyverse)))

In [ ]:
## TRANS

In [2]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/TRANSANCESTRY_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_THREE <- read.table(gzfile("../data/TRANSANCESTRY_GWAS.TAB.gz"),header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_AGEN","PVALUE_METSIM","NA1","NA2","NA3")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
TRANSANCESTRY_RESULTS <- merge(META_THREE,Metasoft_results,by="MARKERNAME")
TRANSANCESTRY_RESULTS <- TRANSANCESTRY_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(AGEN_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+AGEN_EAF*2*AGEN_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(AGEN_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2,-NA3)

In [5]:
lead <- read.table("../../ANALYSIS/lead_transancestry_3mb_merged.txt",fill=TRUE,header=TRUE)
output_df <- merge(TRANSANCESTRY_RESULTS,lead)%>%select(MARKERNAME,CHROMOSOME,POSITION,PVALUE_FE,MAF_AVG)%>%
rename(rsid=MARKERNAME,chromosome=CHROMOSOME,position=POSITION,pvalue=PVALUE_FE,maf=MAF_AVG)%>%mutate(maf = case_when(maf>0.50~ 0.4999,
                            TRUE ~ maf))%>%distinct(rsid,.keep_all = TRUE)

In [6]:
head(output_df)

,rsid,chromosome,position,pvalue,maf
,<chr>,<int>,<int>,<dbl>,<dbl>
1,rs10000081,4,17348363,0.04007360,0.33229026
2,rs10000092,4,21895517,0.00723546,0.38806186
3,rs1000025,6,134085319,0.02211700,0.28441548
4,rs10000252,4,3216484,0.03099270,0.02623173
5,rs10000268,4,63606939,0.06823810,0.16447520
6,rs10000289,4,96437606,0.02300860,0.03235335


In [7]:
lead_variants <- data.frame()

for (i in 1:nrow(lead) ){
   
line_q <- lead[i,]
chr_q <- as.integer(line_q[1])
start_q<- as.integer(line_q[2])
end_q <- as.integer(line_q[3])

tmp_df <- output_df%>%filter(chromosome==chr_q & position >= start_q & position <= end_q)%>% distinct()%>%select(rsid,pvalue)
lead_variants  <- rbind(lead_variants,tmp_df)
}

In [8]:
write.table(lead_variants,"../../ANALYSIS/FUMA/VEGAS2_TRANS_INPUT.TAB",row.names=FALSE,col.names=FALSE,quote=FALSE,sep="\t")

In [ ]:
## EUROPEAN


In [9]:
Metasoft_results <-  read.table("../../ANALYSIS/META/METASOFT/SECOND_PASS/EUROPEAN_METASOFT_SECONDPASS_RESULTS.TAB",fill=TRUE,row.names=NULL,header=FALSE)
META_TWO <- read.table(gzfile("../data/EUR_GWAS.TAB.gz"),header=T,fill=TRUE,row.names=NULL)
colnames(Metasoft_results) <- c("MARKERNAME","N","PVALUE_FE","BETA_FE","STD_FE","PVALUE_RE","BETA_RE","STD_RE","PVALUE_RE2","STAT1_RE2","STAT2_RE2","PVALUE_BE","I_SQUARE","Q","PVALUE_Q","TAU_SQUARE",
                                "PVALUE_ADIPOGEN","PVALUE_METSIM","NA1","NA2")
Metasoft_results <- Metasoft_results%>%filter(MARKERNAME!="RSID")
EUR_RESULTS <- merge(META_TWO,Metasoft_results,by="MARKERNAME")
EUR_RESULTS <- EUR_RESULTS%>%mutate(TOTAL_N=ceiling(ADIPO_N)+ceiling(METSIM_N))%>%mutate(MAF_AVG=(ADIPO_EAF*2*ADIPO_N+METSIM_EAF*2*METSIM_N)/(2*TOTAL_N))%>%
filter(ADIPO_EAF>0.005)%>%filter(METSIM_EAF>0.005)%>%select(-NA1,-N,-NA2)

In [10]:
lead <- read.table("../../ANALYSIS/lead_european_3mb_merged.txt",fill=TRUE,header=TRUE)
 output_df <- merge(EUR_RESULTS,lead)%>%select(MARKERNAME,CHROMOSOME.x,POSITION.x,PVALUE_FE,MAF_AVG)%>%
rename(rsid=MARKERNAME,chromosome=CHROMOSOME.x,position=POSITION.x,pvalue=PVALUE_FE,maf=MAF_AVG)%>%mutate(maf = case_when(maf>0.50~ 0.4999,
                            TRUE ~ maf))%>%distinct(rsid,.keep_all = TRUE)

In [11]:
lead_variants <- data.frame()

for (i in 1:nrow(lead) ){
   
line_q <- lead[i,]
chr_q <- as.integer(line_q[1])
start_q<- as.integer(line_q[2])
end_q <- as.integer(line_q[3])

tmp_df <- output_df%>%filter(chromosome==chr_q & position >= start_q & position <= end_q)%>% distinct()%>%select(rsid,pvalue)
lead_variants  <- rbind(lead_variants,tmp_df)
}

In [12]:
write.table(lead_variants,"../../ANALYSIS/FUMA/VEGAS2_EUR_INPUT.TAB",row.names=FALSE,col.names=FALSE,quote=FALSE,sep="\t")

In [ ]:
## METSIM

In [1]:
METSIM_RESULTS <- read.table("../../Summary_Statistic//Formatted//METSIM_hg19_METAL_POS.TAB",header=F,fill=TRUE,row.names=NULL)
colnames(METSIM_RESULTS) <- c("CHROMOSOME","POSITION","MARKERNAME","METSIM_NEA","METSIM_EA","METSIM_N","PVALUE_FE","METSIM_BETA","METSIM_SE")

In [7]:
lead <- read.table("../../ANALYSIS/lead_metsim_3mb_merged.txt",fill=TRUE,header=TRUE)
output_df <- merge(METSIM_RESULTS,lead)%>%mutate(MAF=0.49)%>%select(MARKERNAME,CHROMOSOME,POSITION,PVALUE_FE)%>%
rename(rsid=MARKERNAME,chromosome=CHROMOSOME,position=POSITION,pvalue=PVALUE_FE)%>%distinct(rsid,.keep_all = TRUE)

In [8]:
lead_variants <- data.frame()

for (i in 1:nrow(lead) ){
   
line_q <- lead[i,]
chr_q <- as.integer(line_q[1])
start_q<- as.integer(line_q[2])
end_q <- as.integer(line_q[3])

tmp_df <- output_df%>%filter(chromosome==chr_q & position >= start_q & position <= end_q)%>% distinct()%>%select(rsid,pvalue)
lead_variants  <- rbind(lead_variants,tmp_df)
}

In [12]:
lead_variants <- na.omit(lead_variants)

In [13]:
write.table(lead_variants,"../../ANALYSIS/FUMA_VEGAS2/METSIM_VEGAS_INPUT.TAB",row.names=FALSE,col.names=FALSE,quote=FALSE,sep="\t")